In [1]:
#%pip install -r requirements.txt

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "NA"

In [3]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3", request_timeout=4000.0)

In [4]:
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

documents = SimpleDirectoryReader("data").load_data()


# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding('BAAI/bge-small-en-v1.5')



c:\datascience\conf_7_nov\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\datascience\conf_7_nov\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\38068\AppData\Local\llama_index\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate develope

In [5]:
import chromadb
from llama_index.core import VectorStoreIndex, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext


In [6]:
# Configure global settings
Settings.llm = llm
Settings.embed_model = embed_model

In [10]:

index = VectorStoreIndex.from_documents(documents, show_progress=True)

Generating embeddings: 100%|██████████| 11/11 [00:04<00:00,  2.25it/s]


In [ ]:
# Set up the retriever from your index
top_k = 3
retriever = index.as_retriever(similarity_top_k=top_k)


'Lowther & SiddikiAIR & SPACE OPERATIONS REVIEW   5defense technologies and began to modernize the military.10 These efforts included coop -\nerating with many Western nations to acquire state- of- the- art military hardware and \nsoftware. In this regard, Ukraine’s homegrown military drone technology was relatively \nyoung when Russia invaded in February 2022.11 Many drones Ukraine has deployed \nagainst Russia were purchased from foreign nations, with Turkey leading the list of sup -\npliers before the start of the war.\nBaykar Bayraktar TB2\nThe Baykar Bayraktar TB2 is a medium- altitude, long- endurance unmanned combat \naerial vehicle that, according to the manufacturer, is capable of remotely controlled or \nautonomous flight operations. This immensely popular drone is manufactured by the \nTurkish defense company Baykar Defense.12 The manufacturer’s primary objective is to \nbuild a less expensive alternative to Western drones, primarily for the Turkish Armed \nForces. Although 

In [19]:
def get_augmented_prompt(query):
    retrieved_documents = retriever.retrieve(query)
    # Augment the prompt by including the retrieved documents
    # Constructing an augmented prompt with context from retrieved documents
    augmented_prompt = f"Context:\n"
    for doc in retrieved_documents:
        text = doc.text
        augmented_prompt += f"{text}\n"
    augmented_prompt += f"\nQuestion: {query}\nPlease provide a concise and accurate answer based on the context."
    return augmented_prompt


In [26]:
from IPython.display import Markdown, display

# Send the augmented prompt to a language model for a synthesized response
def get_RAG_response(query):    
    response = llm.complete(get_augmented_prompt(query))
    return response


In [ ]:
query = "What is the Bayraktar T2?"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))


In [ ]:
query = "Where was used Bayraktar?"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))

## <b>Response:</b> 
 #### According to the text, the Baykar Bayraktar TB2 drone was used in Syria, Libya, and Azerbaijan before its appearance in Ukraine.

In [ ]:

query = "Where was used Bayraktar?"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))

## <b>Response:</b> 
 #### According to the text, before its appearance in Ukraine, the Bayraktar TB2 drone was used in Syria, Libya, and Azerbaijan.

In [29]:

query = "Tell me about Bayraktar practical using"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))

## <b>Response:</b> 
 #### According to the text, the Baykar Bayraktar TB2 is a medium-altitude, long-endurance unmanned combat aerial vehicle (UCAV) manufactured by Turkish defense company Baykar Defense. It has a number of practical uses, including:

* Flying for almost 24 hours at a maximum altitude of 25,000 feet
* Being equipped with four laser-guided bombs, which are proving effective against Russian tanks and other armored vehicles
* Being used in Syria, Libya, and Azerbaijan before being acquired by the Ukrainian military as part of its modernization program
* Proving to be a force multiplier for Ukrainian forces, helping them outmaneuver and outgun Russian troops.

In [30]:

query = "Tell me about Bayraktar practical using"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))

## <b>Response:</b> 
 #### According to the text, the Baykar Bayraktar TB2 is a medium-altitude, long-endurance unmanned combat aerial vehicle (UCAV) manufactured by Turkish defense company Baykar Defense. It can fly for almost 24 hours at a maximum altitude of 25,000 feet and has a wingspan of approximately 39 feet. The TB2 can be equipped with four laser-guided bombs and is commonly used to attack Russian tanks and other armored vehicles. Before its appearance in Ukraine, the drone was used in Syria, Libya, and Azerbaijan. Ukrainian forces have found the Bayraktar to be very effective as a force multiplier.